In [3]:
import gradio as gr
import ollama

In [8]:
system_message = "You are a chat bot , you response in markdown"

def chatllm(question):
    response = ollama.chat(
        model = "qwen2.5:1.5b",
        messages=[{"role": "system", "content":system_message}, {"role":"user", "content":question}]
        # stream=True
    )
    return response.message.content
    

In [13]:
demo = gr.Interface(fn=chatllm,
                    inputs=[gr.Textbox(label="Ask me any question", lines=4)],
                    outputs=[gr.Markdown(label="Response")], 
                    allow_flagging="never")

if __name__ == "__main__":
    demo.launch()


c:\Users\spurusho\Downloads\GenAI\Personal\8WeekCourseLLM\venv\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


# Stream messages

In [ ]:
system_message = "You are a chat bot , you response in markdown format"

def chatllm(question):
    response = ollama.chat(
        model = "qwen2.5:1.5b",
        messages=[{"role": "system", "content":system_message}, {"role":"user", "content":question}],
        stream=True
    )
    result = ""
    for word in response:
        result += word.message.content
        yield result

In [7]:
demo = gr.Interface(fn=chatllm,
                    inputs=[gr.Textbox(label="Ask me any question", lines=4)],
                    outputs=[gr.Markdown(label="Response")], 
                    allow_flagging="never")

if __name__ == "__main__":
    demo.launch()


c:\Users\spurusho\Downloads\GenAI\Personal\8WeekCourseLLM\venv\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [24]:
import requests
from bs4 import BeautifulSoup
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [25]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [36]:
def give_response(response):
    response = response
    result = ""
    for chunk in response:
        result += chunk.message.content
        yield result

In [37]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    result = ollama.chat(
        model = model,
        messages=[{'role':"system", "content": system_message}, {'role': "user", "content":prompt}],
        stream=True
    )
    result = give_response(result)
    yield from result

In [38]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["qwen2.5:1.5b", "deepseek-r1:1.5"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
